In [8]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

In [9]:
from pydantic import BaseModel
from typing import List


class SubSection(BaseModel):
    title: str
    content: str


class BlogPost(BaseModel):
    title: str
    introduction: str
    sections: List[SubSection]
    sources: List[str]
    hashtags: List[str]

In [12]:
from crewai import Crew, Agent, Task
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

researcher = Agent(
    role="Senior Researcher",
    goal="Search the web, extract and analyze information.",
    backstory="""
    You produce the highest quality research possible.
    You use multiple sources of information and you always double check your sources to make sure they are true and up to date.
    You want to impress your coworkers with your work.
    """,
    allow_delegation=False,
    verbose=True,
    tools=[
        search_tool,
        scrape_tool,
    ],
    max_iter=10,
)

editor = Agent(
    role="Senior Writer/Editor",
    goal="Write engaging blog posts.",
    backstory="""
    You write content that keeps people engaged and entertained.
    Your content is easy to read it is informative and it makes people want to share it with their friends.
    You are working for a very important client.
    """,
    allow_delegation=True,
    verbose=True,
)

task = Task(
    description="Write a blog post about {topic}",
    agent=editor,
    expected_output="A blog post with an introduction, at least three sub-sections of content, links to sources, a set of suggested hashtags for social media and a catchy title.",
    output_file="blog_post.md",
    # output_pydantic=BlogPost,
    output_json=BlogPost,
)

crew = Crew(
    agents=[
        researcher,
        editor,
    ],
    tasks=[task],
    verbose=2,
)

result = crew.kickoff(
    inputs=dict(
        topic="The biggest box office flops of 2024",
    )
)

2024-10-09 21:59:53,618 - 22768 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Senior Writer/Editor
 [INFO]: == Starting Task: Write a blog post about The biggest box office flops of 2024


> Entering new CrewAgentExecutor chain...
I need to gather information about the biggest box office flops of 2024 to write an engaging blog post. Since it's early in the year, I will delegate the task of researching the relevant films and their box office performances to a co-worker who specializes in research. 

Action: Delegate work to co-worker
Action Input: {
    "task": "Research and compile a list of the biggest box office flops of 2024, including details such as the film titles, budgets, box office earnings, and reasons for their poor performance.",
    "context": "The blog post aims to inform readers about major films that underperformed at the box office in 2024. This will include an introduction to the topic, at least three sub-sections discussing different films, and an analysis of why these films did not succeed. The information should b